In [1]:
from GeneralTools.misc_fun import FLAGS

FLAGS.TENSORFLOW_VERSION


'1.13.0'

In [9]:
import tensorflow as tf
"""
Scope - continue

tf.variable_scope vs tf.name_scope
 -- variable_scope applies to constants, variables and operations
 -- name_scope applies to constants and operations, but not variables
"""

with tf.Graph().as_default():
    a1 = tf.constant(0.1, name='const_a1')
    b1 = tf.get_variable(
        name='var_b1', dtype=tf.float32, 
        initializer=a1)
    c1 = tf.multiply(a1, b1, name='op_1')

    with tf.variable_scope('layer01'):
        # variable scope applies to constants, variables and operations
        a2 = tf.constant(0.1, name='const_a2')
        b2 = tf.get_variable(
            name='var_b2', dtype=tf.float32, 
            initializer=a1)  # initializer could be a1, b1, a2
        c2 = tf.multiply(a2, b2, name='op_2')
        
    with tf.name_scope('layer02'):
        # name scope applies to constants and operations
        a3 = tf.constant(0.1, name='const_a3')
        # name scope does not apply to variables
        b3 = tf.get_variable(
            name='var_b3', dtype=tf.float32, 
            initializer=a1)  # initialize could be a1, b1, a2, b2, a3
        c3 = tf.multiply(a3, b3, name='op_3')
    
print("The name/operation of a1 is '{}'".format(a1.name))
print("The name/operation of b1 is '{}'".format(b1.name))
print("The name/operation of c1 is '{}'".format(c1.name))
print('------')
print("The name/operation of a2 is '{}'".format(a2.name))
print("The name/operation of b2 is '{}'".format(b2.name))
print("The name/operation of c2 is '{}'".format(c2.name))
print('------')
print("The name/operation of a3 is '{}'".format(a3.name))
print("The name/operation of b3 is '{}'".format(b3.name))
print("The name/operation of c3 is '{}'".format(c3.name))


The name/operation of a1 is 'const_a1:0'
The name/operation of b1 is 'var_b1:0'
The name/operation of c1 is 'op_1:0'
------
The name/operation of a2 is 'layer01/const_a2:0'
The name/operation of b2 is 'layer01/var_b2:0'
The name/operation of c2 is 'layer01/op_2:0'
------
The name/operation of a3 is 'layer02/const_a3:0'
The name/operation of b3 is 'var_b3:0'
The name/operation of c3 is 'layer02/op_3:0'


In [6]:
import tensorflow as tf
from my_classifier import MyClassifier

"""
Train a model

Here I used debug model to do profiling / tracing at the last five steps
I also loaded previously saved model and save the model after training.
"""

with tf.Graph().as_default():
    # load the data
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    
    # define the model
    net_design = [{'name': 'l1', 'out': 33, 'op': 'd', 'act': 'relu', 'in_reshape': [784]}, 
                  {'name': 'l2', 'out': 66, 'op': 'd', 'act': 'relu'}, 
                  {'name': 'l3', 'out': 10, 'op': 'd'}]
    mdl = MyClassifier(
        net_design, input_shape=[None, 28, 28], num_class=10, 
        optimizer='momentum', name='classifier')
    # do training use debug model
    # for long training process, turn off debug mode and increase max_step
    mdl.train(
        x_train, y_train, batch_size=60,
        map_func=lambda x, y: (x/255.0, y), 
        x_validate=x_test, y_validate=y_test, 
        model_folder='trial/trial_01', max_step=100, 
        debug_mode=True, query_step=20, load_ckpt=True, do_trace=True)


TrainData dataset output type is (tf.float32, tf.int32)
TrainData dataset output shape is (TensorShape([Dimension(28), Dimension(28)]), TensorShape([]))
ValidateData dataset output type is (tf.float32, tf.int32)
ValidateData dataset output shape is (TensorShape([Dimension(28), Dimension(28)]), TensorShape([]))
Momentum Optimizer is used.


Graph initialization finished...
No ckpt model is loaded for current calculation.


No ckpt model is loaded for current calculation.


INFO:tensorflow:Restoring parameters from C:/Users/richa/PycharmProjects/ai_mechatronics/Results/trial/trial_01_ckpt\trial_01.ckpt-100


Model reloaded from C:/Users/richa/PycharmProjects/ai_mechatronics/Results/trial/trial_01_ckpt\.


Epoch 0, global steps 101, training loss: <1.135>, accuracy: <0.683>, validation loss: <1.078>, acc: <0.710>
Epoch 0, global steps 121, training loss: <0.536>, accuracy: <0.900>, validation loss: <0.360>, acc: <0.930>


Epoch 0, global steps 141, training loss: <0.407>, accuracy: <0.883>, validation loss: <0.346>, acc: <0.920>
Epoch 0, global steps 161, training loss: <0.338>, accuracy: <0.900>, validation loss: <0.297>, acc: <0.920>


Epoch 0, global steps 181, training loss: <0.327>, accuracy: <0.917>, validation loss: <0.356>, acc: <0.870>


Training for 100 steps took 1.734 sec.
Session finished.


In [5]:
import tensorflow as tf
from my_classifier import MyClassifier

"""
Test a model

Here I reinitialized the model, loaded previously saved model and 
did predictions on new test data.
"""

with tf.Graph().as_default():
    # load the data
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    
    # define the model
    net_design = [{'name': 'l1', 'out': 33, 'op': 'd', 'act': 'relu', 'in_reshape': [784]}, 
                  {'name': 'l2', 'out': 66, 'op': 'd', 'act': 'relu'}, 
                  {'name': 'l3', 'out': 10, 'op': 'd'}]
    mdl = MyClassifier(
        net_design, input_shape=[None, 28, 28], num_class=10, 
        optimizer='momentum', name='classifier')
    # make prediction
    predictions, loss, acc = mdl.test(
        x_test, y_test, map_func=lambda x, y: (x/255.0, y), model_folder='trial/trial_01')
    
    print('The test accuracy on test set is {:.3f}'.format(acc))


TestData dataset output type is (tf.float32, tf.int32)
TestData dataset output shape is (TensorShape([Dimension(28), Dimension(28)]), TensorShape([]))
Graph initialization finished...
No ckpt model is loaded for current calculation.


INFO:tensorflow:Restoring parameters from C:/Users/richa/PycharmProjects/ai_mechatronics/Results/trial/trial_01_ckpt\trial_01.ckpt-100


Model reloaded from C:/Users/richa/PycharmProjects/ai_mechatronics/Results/trial/trial_01_ckpt\.
Calculation took 0.031 sec.
Session finished.
The test accuracy is 0.560


In [2]:
import tensorflow as tf
from my_classifier import MyClassifier

"""
Visualize any layer output

"""

with tf.Graph().as_default():
    # load the data
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    
    # define the model
    net_design = [{'name': 'l1', 'out': 33, 'op': 'd', 'act': 'relu', 'in_reshape': [784]}, 
                  {'name': 'l2', 'out': 66, 'op': 'd', 'act': 'relu'}, 
                  {'name': 'l3', 'out': 10, 'op': 'd'}]
    mdl = MyClassifier(
        net_design, input_shape=[None, 28, 28], num_class=10, 
        optimizer='momentum', name='classifier')
    
    layer_to_visualize = 'l2'  # could be 'input' to visualize the raw data
    mdl.visualize_layer_output(
        layer_to_visualize, x_train, y_train, batch_size=400, 
        map_func=lambda x, y: (x/255.0, y),
        model_folder='trial/trial_01', 
        save_folder='trial/visual_{}/{}'.format(layer_to_visualize, layer_to_visualize), 
        invert_image=True)


VisualizeData dataset output type is (tf.float32, tf.int32)
VisualizeData dataset output shape is (TensorShape([Dimension(28), Dimension(28)]), TensorShape([]))


Graph initialization finished...
No ckpt model is loaded for current calculation.


INFO:tensorflow:Restoring parameters from C:/Users/richa/PycharmProjects/ai_mechatronics/Results/trial/trial_01_ckpt\trial_01.ckpt-200


Model reloaded from C:/Users/richa/PycharmProjects/ai_mechatronics/Results/trial/trial_01_ckpt\.


Session finished.
Mesh_num will be calculated as sqrt of batch_size
